In [2]:
import requests 
import urllib.request
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print(url.text[0:500])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","w


Comment: First step is to load the web content to url

# lxml installation to able to handle "url" content

In [3]:
!conda install -c anaconda lxml

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |           py35_1         139 KB  anaconda
    ca-certificates-2019.1.23  |                0         126 KB  anaconda
    openssl-1.0.2r             |       h7b6447c_0         3.2 MB  anaconda
    libxml2-2.9.9              |       he19cac6_0         2.0 MB  anaconda
    libxslt-1.1.33             |       h7d1a2b0_0         577 KB  anaconda
    lxml-4.2.5                 |   py35hefd8a0e_0         1.6 MB  anaconda
    ------------------------------------------------------------
                                           Total:         7.6 MB

The following packages will be UPDATED:

    ca-certificates: 2019.1.23-0          --> 2019.1.23-0          anaconda
    certifi:      

In [4]:
from html.parser import HTMLParser

# # In this part we will extarct the postal table from web loaded data

In [5]:
#with open ('List_of_postal_codes_of_Canada:_M','https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') as file:
lien=url.text
from lxml import html as lh

In [6]:
import pandas as pd
import numpy as np

In [7]:
doc = lh.fromstring(url.content)

In [8]:
tr_elements = doc.xpath('//tr')

In [9]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [10]:
tr_elements = doc.xpath('//tr')
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [11]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [12]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [13]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

# == > Data Formating and clean up as required in the exercice

In [14]:
df_final=pd.DataFrame()
df_final['Postcode'] = df['Postcode']
df_final['Borough'] = df['Borough']
df_final['Neighbourhood'] = df.iloc[ : , 1 ]
df_final=df_final[df_final.Borough != 'Not assigned']

In [15]:
df_final=df_final.replace('\n', '', regex=True)
df_final.loc[df_final.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df_final.Borough
df_final.reset_index(drop=True)
df_final.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [16]:
df_test=df_final

In [17]:
df_final_last=df_test.groupby(['Postcode','Borough'],as_index=False).agg(lambda x :', '.join(x))

In [22]:
df_final_last.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [20]:
df_final_last.describe()




,Postcode,Borough,Neighbourhood
count,103,103,103
unique,103,11,103
top,M5B,North York,Central Bay Street
freq,1,24,1


In [21]:
df_final_last.shape

(103, 3)

# ==> Import CSV file for coordinates

In [24]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Insert coordiante to Neighbhood dataframe

In [31]:
df_data_1.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_final_last=df_final_last.merge(df_data_1, on='Postcode', how='left')

In [32]:
df_final_last.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [33]:
df_final_last.shape

(103, 5)